In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchvision
import torchvision.transforms as transforms
from opacus.validators import ModuleValidator

In [2]:
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager

In [3]:
def create_resnet18(num_classes=10):
    model = torchvision.models.resnet18(num_classes=num_classes)
    errors = ModuleValidator.validate(model, strict=False)
    errors[-5:]
    model = ModuleValidator.fix(model)
    ModuleValidator.validate(model, strict=False)
    return model

In [4]:
# model = torchvision.models.resnet18(num_classes=10)

In [5]:
# errors = ModuleValidator.validate(model, strict=False)
# errors[-5:]

In [6]:
# model = ModuleValidator.fix(model)
# ModuleValidator.validate(model, strict=False)

In [7]:
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)
Batch_sz=512

In [8]:
def load_data(num_clients,MEAN,STD_DEV,Batch_Size):
    transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(MEAN,STD_DEV),])

    train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
    test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform)
    print(len(train_dataset))

    data_size=len(train_dataset)//num_clients
    client_datasets=random_split(train_dataset,[data_size]*num_clients)

    client_loaders=[DataLoader(ds,batch_size=Batch_Size,shuffle=True) for ds in client_datasets]
    test_loader=DataLoader(test_dataset, batch_size=Batch_Size, shuffle=False)
    
    return client_loaders, test_loader


In [9]:
MAX_GRAD_NORM = 1.2
EPSILON = 60.0
DELTA = 1e-5
EPOCHS = 150
MAX_PHYSICAL_BATCH_SIZE = 64
LR = 1e-2

In [10]:
# def make_private(model,train_loader,ePOCHS,ePSILON,dELTA,mAX_GRAD_NORM):
#     privacy_engine = PrivacyEngine()
#     optimizer = optim.RMSprop(model.parameters(), lr=LR)
#     model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
#         module=model,
#         optimizer=optimizer,
#         data_loader=train_loader,
#         epochs=ePOCHS,
#         target_epsilon=ePSILON,
#         target_delta=dELTA,
#         max_grad_norm=mAX_GRAD_NORM,
#     )
#     print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
#     return model, optimizer, train_loader


In [11]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [12]:
def train_client(model, train_loader, ePSILON,dELTA,mAX_GRAD_NORM,epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []

    privacy_engine = PrivacyEngine()
    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        epochs=epoch,
        target_epsilon=ePSILON,
        target_delta=dELTA,
        max_grad_norm=mAX_GRAD_NORM,
    )
    print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")
    
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):   
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.2f}, δ = {DELTA})"
                )

In [13]:
def evaluate(model, test_loader, device="cuda"):
    model.eval()
    test_loss = 0
    correct = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss +=criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    return test_loss, accuracy

In [14]:
# def average_models1(models,device):
#     averaged_model = create_resnet18()

#     averaged_model = averaged_model.to(device)
    
#     with torch.no_grad():
#         for param in averaged_model.parameters():
#             param.data.zero_()
            
#         for model in models:
#             for param_avg, param_model in zip(averaged_model.parameters(), model.parameters()):
#                 param_avg.data += param_model.data
                
#         # Divide by number of models
#         for param in averaged_model.parameters():
#             param.data /= len(models)
    
#     return averaged_model

In [15]:
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD_DEV = (0.2023, 0.1994, 0.2010)
Batch_sz=512

In [16]:
def paramet_stor(client_models,client_loaders,device,num_clients=2,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz,ePOCHS=EPOCHS):
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # client_loaders, test_loader = load_data(num_clients=num_clients,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz)
    # client_models = [create_resnet18().to(device) for _ in range(num_clients)]

    client_parameters = []
    for client_id, (model, loader) in enumerate(zip(client_models, client_loaders)):
        print(f"\nTraining Client {client_id + 1}")
        # model, optimizer, loader=make_private(model,loader,ePOCHS=EPOCHS,ePSILON=EPSILON,dELTA=DELTA,mAX_GRAD_NORM=MAX_GRAD_NORM)
        train_client(model=model, train_loader=loader, epoch=EPOCHS,ePSILON=EPSILON,dELTA=DELTA,mAX_GRAD_NORM=MAX_GRAD_NORM,device=device)
        client_parameters.append([param.data.clone() for param in model.parameters()])
        for param in model.parameters():
            param.data.zero_()
    return client_parameters, client_models


In [17]:
def average_models(models,client_parameters):
    averaged_model = create_resnet18()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    averaged_model = averaged_model.to(device)
    
    with torch.no_grad():
        for param in averaged_model.parameters():
            param.data.zero_()
            
        for param_avg, param_list in zip(averaged_model.parameters(), zip(*client_parameters)):
            param_avg.data.copy_(sum(p for p in param_list) / len(models))
    
    return averaged_model

In [18]:
# client_params, models = paramet_stor()
 

In [19]:
# averaged_model = average_models(models,client_params)

In [20]:
def federated_learning(num_clients=10, num_rounds=5,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz,ePOCHS=EPOCHS,ePSILON=EPSILON,dELTA=DELTA,mAX_GRAD_NORM=MAX_GRAD_NORM):
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Load data
    client_loaders, test_loader = load_data(num_clients=num_clients,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz)
    
    # Initialize models
    
    super_model = create_resnet18().to(device)
    
    # Track metrics
    round_accuracies = []
    
    for round in range(num_rounds):
        print(f"\n=== Federated Learning Round {round + 1}/{num_rounds} ===")

        client_models = [create_resnet18().to(device) for _ in range(num_clients)]
        for model in client_models:
            model.load_state_dict(super_model.state_dict())
        
        # Train each client model
        client_params, client_models=paramet_stor(client_models=client_models,client_loaders=client_loaders,device=device,num_clients=10,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz,ePOCHS=EPOCHS)
            
        # Average models
        super_model = average_models( client_models,client_params)
        
        # Evaluate super model
        test_loss, test_accuracy = evaluate(super_model, test_loader, device)
        round_accuracies.append(test_accuracy)
        print(f"\nSuper Model - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        
        # # Update client models with averaged weights
        for model in client_models:
            model.load_state_dict(super_model.state_dict())
        
        # Evaluate individual clients
        print("\nClient Models Performance:")
        for client_id, model in enumerate(client_models):
            loss, accuracy = evaluate(model, test_loader, device)
            print(f"Client {client_id + 1} - Test Accuracy: {accuracy:.2f}%")
        del client_models
    # Print final results
    print("\n=== Final Results ===")
    print("Round-wise Super Model Accuracies:")
    for round, acc in enumerate(round_accuracies, 1):
        print(f"Round {round}: {acc:.2f}%")
    print(f"Final Super Model Accuracy: {round_accuracies[-1]:.2f}%")

In [21]:
# def federated_learning(num_clients=10, num_rounds=5,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz,ePOCHS=EPOCHS,ePSILON=EPSILON,dELTA=DELTA,mAX_GRAD_NORM=MAX_GRAD_NORM):
#     # Set device
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {device}")
    
#     # Load data
#     client_loaders, test_loader = load_data(num_clients=num_clients,MEAN=CIFAR10_MEAN,STD_DEV=CIFAR10_STD_DEV,Batch_Size=Batch_sz)
    
#     # Initialize models
#     client_models = [create_resnet18().to(device) for _ in range(num_clients)]
#     super_model = create_resnet18().to(device)
    
#     # Track metrics
#     round_accuracies = []
    
#     for round in range(num_rounds):
#         print(f"\n=== Federated Learning Round {round + 1}/{num_rounds} ===")
        
#         # Train each client model
#         for client_id, (model, loader) in enumerate(zip(client_models, client_loaders)):
#             print(f"\nTraining Client {client_id + 1}")
#             model, optimizer, loader=make_private(model,loader,ePOCHS=EPOCHS,ePSILON=EPSILON,dELTA=DELTA,mAX_GRAD_NORM=MAX_GRAD_NORM)
#             train_client(model, loader, optimizer, EPOCHS,device)
            
            
#         # Average models
#         super_model = average_models(client_models,device)
        
#         # Evaluate super model
#         test_loss, test_accuracy = evaluate(super_model, test_loader, device)
#         round_accuracies.append(test_accuracy)
#         print(f"\nSuper Model - Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")
        
#         # Update client models with averaged weights
#         for model in client_models:
#             model.load_state_dict(super_model.state_dict())
        
#         # Evaluate individual clients
#         print("\nClient Models Performance:")
#         for client_id, model in enumerate(client_models):
#             loss, accuracy = evaluate(model, test_loader, device)
#             print(f"Client {client_id + 1} - Test Accuracy: {accuracy:.2f}%")
    
#     # Print final results
#     print("\n=== Final Results ===")
#     print("Round-wise Super Model Accuracies:")
#     for round, acc in enumerate(round_accuracies, 1):
#         print(f"Round {round}: {acc:.2f}%")
#     print(f"Final Super Model Accuracy: {round_accuracies[-1]:.2f}%")

In [22]:
if __name__ == "__main__":
    federated_learning(num_clients=2, num_rounds=3)

Using device: cuda
Files already downloaded and verified


50000

=== Federated Learning Round 1/3 ===

Training Client 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


Using sigma=0.5458641052246094 and C=1.2


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/nn/modules/module.py:1352: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


	Train Epoch: 150 	Loss: 24.783700 Acc@1: 9.524052 (ε = 5.36, δ = 1e-05)
	Train Epoch: 150 	Loss: 14.510549 Acc@1: 9.965573 (ε = 6.25, δ = 1e-05)

Training Client 2
Using sigma=0.5458641052246094 and C=1.2
	Train Epoch: 150 	Loss: 24.164040 Acc@1: 9.869386 (ε = 5.36, δ = 1e-05)
	Train Epoch: 150 	Loss: 14.354968 Acc@1: 10.032081 (ε = 6.29, δ = 1e-05)

Super Model - Test Loss: 0.0053, Test Accuracy: 10.00%

Client Models Performance:
Client 1 - Test Accuracy: 10.00%
Client 2 - Test Accuracy: 10.00%

=== Federated Learning Round 2/3 ===

Training Client 1
Using sigma=0.5458641052246094 and C=1.2
	Train Epoch: 150 	Loss: 9.884464 Acc@1: 10.094872 (ε = 5.36, δ = 1e-05)
	Train Epoch: 150 	Loss: 6.137544 Acc@1: 9.988227 (ε = 6.29, δ = 1e-05)

Training Client 2
Using sigma=0.5458641052246094 and C=1.2
	Train Epoch: 150 	Loss: 10.360314 Acc@1: 10.109482 (ε = 5.36, δ = 1e-05)
	Train Epoch: 150 	Loss: 6.366293 Acc@1: 10.155838 (ε = 6.25, δ = 1e-05)

Super Model - Test Loss: 0.0046, Test Accuracy

In [23]:
# if __name__ == "__main__":
#     federated_learning(num_clients=10, num_rounds=5)